In [46]:
from __future__ import division
import numpy as np
import sys

In [47]:
X_train = np.genfromtxt('X_train_C.csv', delimiter=',')
y_train = np.genfromtxt('Y_train_C.csv', delimiter=',')
X_test = np.genfromtxt('X_test_C.csv', delimiter=',')

In [48]:
# X_train = np.array([[1, 1, 2, 3], [2, 7, -4, 8], [6, 5, 4, 2], [4, -8, -1, 0], [-1, -3, 1, 4], [-8, -2, 3, 1], [4, -1, 2, 7]])
# y_train = np.array([0., 1., 1., 0., 1., 1., 0.])
    
# X_test = np.array([[2, 4, 7, 1], [-1, 2, -4, 3], [4, -1, 3, 9], [-2, 4, -1, 6], [3, -3, 4, 2]])

In [49]:
Y, n_y = np.unique(y_train, return_counts=True)

In [50]:
print(Y, n_y)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.] [41 47 42 37 35 34 42 23 32 17]


In [51]:
pi_hat = []
mu_hat = []
sigma_hat = []
for y, count_y in zip(Y, n_y):
    
    pi_hat_y = count_y/np.sum(n_y)
    pi_hat.append(pi_hat_y)
    mu_hat_y = np.dot((y_train == y), X_train)/count_y
    mu_hat.append(mu_hat_y)
    X_train_y = X_train[y_train.astype(int) == np.int(y), :]
    
    x_mu_hat_dev_y = X_train_y - mu_hat_y
        
    sigma_hat_y = np.dot(x_mu_hat_dev_y.T, x_mu_hat_dev_y)/(count_y)
    
    sigma_hat.append(sigma_hat_y)

In [74]:
# predicting for X_test
y_test = []
    
for x in X_test:
    
    f_x_hat = []
    for y in Y:
        idx = np.int(y)
        x_mu_hat_dev_y = x - mu_hat[idx]

        coefficient = pi_hat[idx]/np.sqrt(np.linalg.det(sigma_hat[idx]))
        
        power_value = -(1/2)*(x_mu_hat_dev_y.T@np.linalg.inv(sigma_hat[idx])@x_mu_hat_dev_y)
 
        f_x_hat_y = coefficient*np.exp(power_value)
        f_x_hat.append(f_x_hat_y)

    y_test.append(f_x_hat/np.asarray(f_x_hat).sum())

print(np.asarray(y_test).shape)

(42, 10)


In [75]:
np.savetxt("probs_test.csv", np.asarray(y_test), delimiter=",") # write output to file